In [1]:
import numpy as np 
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings('ignore')

In [2]:
X ,y = make_classification(n_samples=1000,n_features=10,n_informative=2,n_redundant=8,
                          weights=[0.9,0.1],flip_y=0,random_state=42)

np.unique(y,return_counts=True) # Imbalabce class (900,100)

(array([0, 1]), array([900, 100]))

In [3]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,stratify=y,random_state=42)

# Experiment 1 :Train Logistic Regression Classifier

In [4]:
log_reg = LogisticRegression(C=1,solver='liblinear')
log_reg.fit(X_train,y_train)
y_pred_log_reg = log_reg.predict(X_test)
print(classification_report(y_test,y_pred_log_reg))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95       270
           1       0.60      0.50      0.55        30

    accuracy                           0.92       300
   macro avg       0.77      0.73      0.75       300
weighted avg       0.91      0.92      0.91       300



# Experiment 2 :Train Random Forest Classifier

In [5]:
rf_clf = RandomForestClassifier(n_estimators=30,max_depth=3)
rf_clf.fit(X_train,y_train)
y_pred_rf = rf_clf.predict(X_test)
print(classification_report(y_test,y_pred_rf))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98       270
           1       0.95      0.67      0.78        30

    accuracy                           0.96       300
   macro avg       0.96      0.83      0.88       300
weighted avg       0.96      0.96      0.96       300



# Experiment 3 :Train XGBBoost

In [6]:
xgb_clf = XGBClassifier(use_label_encoder=False,eval_metric ='logloss')
xgb_clf.fit(X_train,y_train)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test,y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       270
           1       0.96      0.80      0.87        30

    accuracy                           0.98       300
   macro avg       0.97      0.90      0.93       300
weighted avg       0.98      0.98      0.98       300



# Experiment 4 : Handle class imbalance using SMOTE Tomek and then Train XGBoost

In [7]:
#conda install -c conda-forge scikit-learn=1.4 imbalanced-learn=0.12

In [8]:
import sklearn, imblearn
print("sklearn:", sklearn.__version__)
print("imblearn:", imblearn.__version__)

sklearn: 1.4.2
imblearn: 0.12.4


In [9]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res , y_train_res = smt.fit_resample(X_train,y_train)

np.unique(y_train_res,return_counts=True) # Balanced the class

(array([0, 1]), array([619, 619]))

In [10]:
xgb_clf = XGBClassifier(use_label_encoder=False,eval_metric ='logloss')
xgb_clf.fit(X_train_res,y_train_res)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test,y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       270
           1       0.81      0.83      0.82        30

    accuracy                           0.96       300
   macro avg       0.89      0.91      0.90       300
weighted avg       0.96      0.96      0.96       300



# Track Experiments Using MLFlow

In [11]:
models = [
    (
        "Logistic Regression",
        LogisticRegression(C=1,solver='liblinear'),
        (X_train,y_train), 
        (X_test,y_test)
    ),
    (
        "Random Forest", 
        RandomForestClassifier(n_estimators=30,max_depth=3),
        (X_train,y_train), 
        (X_test,y_test)
    ),
    (
        "XGBClassifier",
        XGBClassifier(use_label_encoder=False, eval_metric = 'logloss'), 
        (X_train,y_train), 
        (X_test,y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        XGBClassifier(use_label_encoder=False,eval_metric='logloss'), 
        (X_train,y_train), 
        (X_test,y_test)
    ),
]

In [12]:
reports = []

for model_name , model ,train_set ,test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]

    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test,y_pred , output_dict= True)
    reports.append(report)

reports

[{'0': {'precision': 0.9454545454545454,
   'recall': 0.9629629629629629,
   'f1-score': 0.9541284403669725,
   'support': 270.0},
  '1': {'precision': 0.6,
   'recall': 0.5,
   'f1-score': 0.5454545454545454,
   'support': 30.0},
  'accuracy': 0.9166666666666666,
  'macro avg': {'precision': 0.7727272727272727,
   'recall': 0.7314814814814814,
   'f1-score': 0.749791492910759,
   'support': 300.0},
  'weighted avg': {'precision': 0.9109090909090909,
   'recall': 0.9166666666666666,
   'f1-score': 0.91326105087573,
   'support': 300.0}},
 {'0': {'precision': 0.96415770609319,
   'recall': 0.9962962962962963,
   'f1-score': 0.9799635701275046,
   'support': 270.0},
  '1': {'precision': 0.9523809523809523,
   'recall': 0.6666666666666666,
   'f1-score': 0.7843137254901961,
   'support': 30.0},
  'accuracy': 0.9633333333333334,
  'macro avg': {'precision': 0.9582693292370712,
   'recall': 0.8314814814814815,
   'f1-score': 0.8821386478088503,
   'support': 300.0},
  'weighted avg': {'prec

In [13]:
import mlflow

In [14]:
client = mlflow.tracking.MlflowClient()
client.search_experiments(view_type=2)   # view_type=2 → include deleted


[]

In [15]:
# Find the deleted experiment
client = mlflow.tracking.MlflowClient()
for exp in client.search_experiments(view_type=2):
    print(exp.experiment_id, exp.name, exp.lifecycle_stage)


In [16]:
# Restore the deleted experiment 
'''client.restore_experiment(experiment_id)
mlflow.set_experiment("Anomaly Detection")'''


'client.restore_experiment(experiment_id)\nmlflow.set_experiment("Anomaly Detection")'

In [19]:
mlflow.set_experiment("Anomaly Detection2")
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

In [20]:


for i , element in enumerate(models):
    model_name = element[0]
    model = element[1]
    report = reports[i]

    with mlflow.start_run(run_name=model_name): 
        mlflow.log_params({'model_name': model_name})
        mlflow.log_metric('accuracy',report['accuracy'])
        mlflow.log_metric('recall_class_1',report['1']['recall'])
        mlflow.log_metric('recall_class_0',report['0']['recall'])
        mlflow.log_metric('f1_score_macro',report['macro avg']['f1-score'])

        if'XGB' in model_name:
            mlflow.xgboost.log_model(model, 'model')
        else:
            mlflow.sklearn.log_model(model,'model')

2025/11/14 13:35:48 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Logistic Regression at: http://127.0.0.1:5000/#/experiments/549924877729902516/runs/5a60ffee903742299a4ba5f03c6c8d47
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/549924877729902516


2025/11/14 13:35:55 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Random Forest at: http://127.0.0.1:5000/#/experiments/549924877729902516/runs/5681be26c5b04f8f888d6b354e928574
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/549924877729902516


2025/11/14 13:36:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier at: http://127.0.0.1:5000/#/experiments/549924877729902516/runs/9301c7a37fb34688b63043b4d7f59b10
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/549924877729902516


2025/11/14 13:36:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier With SMOTE at: http://127.0.0.1:5000/#/experiments/549924877729902516/runs/beb11c04553446c1b37188040dae6c70
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/549924877729902516


# Model Registry

# Track Experiments 

In [21]:
models = [
    (
        "Logistic Regression",
        {"C":1, "solver":'liblinear'}, # parameters
        LogisticRegression(),
        (X_train,y_train), 
        (X_test,y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators":30, "max_depth":3},
        RandomForestClassifier(),
        (X_train,y_train), 
        (X_test,y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder":False, "eval_metric":'logloss'},
        XGBClassifier(), 
        (X_train,y_train), 
        (X_test,y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder":False,"eval_metric":'logloss'},
        XGBClassifier(), 
        (X_train,y_train), 
        (X_test,y_test)
    ),
]

In [22]:
reports = []

for model_name ,params, model ,train_set ,test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]

    model.set_params(**params)
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test,y_pred , output_dict= True)
    reports.append(report)

In [23]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [24]:
# Initialize MLflow
mlflow.set_experiment("Anomaly Detection2")
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000/")

for i , element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]

    with mlflow.start_run(run_name=model_name): 
        mlflow.log_params(params)
        mlflow.log_metrics({
    'accuracy': report['accuracy'],
    'recall_class_1': report['1']['recall'],
    'recall_class_0': report['0']['recall'],
    'f1_score_macro': report['macro avg']['f1-score']
})
        if'XGB' in model_name:
            mlflow.xgboost.log_model(model, 'model')
        else:
            mlflow.sklearn.log_model(model,'model')

2025/11/14 13:36:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Logistic Regression at: http://127.0.0.1:5000/#/experiments/549924877729902516/runs/02aa6561b7084c1fadf9858da1d2da5e
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/549924877729902516


2025/11/14 13:36:39 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Random Forest at: http://127.0.0.1:5000/#/experiments/549924877729902516/runs/70a4be0e6e9f476dbda3b12a2a3e6ba1
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/549924877729902516


2025/11/14 13:36:46 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier at: http://127.0.0.1:5000/#/experiments/549924877729902516/runs/1d85af30c1cf417e85025d5ba217c1df
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/549924877729902516


2025/11/14 13:36:53 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier With SMOTE at: http://127.0.0.1:5000/#/experiments/549924877729902516/runs/c8b2d486300749ff8cf7db58b654c525
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/549924877729902516


# Register the model

In [34]:
# API , copy run id from experiments , Re-run to get new version

model_name = "XGB_with_smote"
run_id = input("Enter Run ID:")
model_uri = f"runs:/{run_id}/model"

result = mlflow.register_model(
    model_uri, model_name
)

Enter Run ID: c8b2d486300749ff8cf7db58b654c525


Registered model 'XGB_with_smote' already exists. Creating a new version of this model...
2025/11/14 13:51:46 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB_with_smote, version 4
Created version '4' of model 'XGB_with_smote'.


# Load the model

In [38]:
# Check available aliases for the model
from mlflow.tracking import MlflowClient

client = MlflowClient()
versions = client.get_latest_versions(model_name)  # shows all stages
for v in versions:
    print(v.version, v.current_stage)


4 None


In [37]:
model_name = "XGB_with_smote"
model_version = 1
model_uri = f"models:/{model_name}@challenger"

loaded_model = mlflow.xgboost.load_model(model_uri) # replace as per your model
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

In [39]:
# Transition from Development environment to production

dev_model_uri = f"models:/{model_name}@challenger"
prod_model = "anomaly-detection-prod"

client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri=dev_model_uri,dst_name=prod_model)

Successfully registered model 'anomaly-detection-prod'.
Copied version '2' of model 'XGB_with_smote' to version '1' of model 'anomaly-detection-prod'.


<ModelVersion: aliases=[], creation_timestamp=1763108901767, current_stage='None', description='', last_updated_timestamp=1763108901767, name='anomaly-detection-prod', run_id='c8b2d486300749ff8cf7db58b654c525', run_link='', source='models:/XGB_with_smote/2', status='READY', status_message=None, tags={}, user_id='', version='1'>

In [40]:
model_uri = f"models:/{prod_model}@champion"

loaded_model = mlflow.xgboost.load_model(model_uri) # replace as per your model
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])